In [29]:
import pandas as pd

Import dos datasets

In [30]:
df_bts = pd.read_csv('input_data/01_raw_files/Bts.csv')
df_test_alunos = pd.read_csv('input_data/01_raw_files/meds_test_alunos.csv')
df_train_alunos = pd.read_csv('input_data/01_raw_files/meds_train_alunos.csv')

Tratamento inicial dos dados

In [31]:
numeric_columns = [
    'rssi_1_1', 'rssi_1_2', 'rssi_1_3', 'rssi_2_1', 'rssi_2_2',
    'rssi_2_3', 'rssi_3_1', 'rssi_3_2', 'rssi_3_3', 'lat', 'lon',
    'delay_1',  'delay_2',  'delay_3'
]

for col in numeric_columns:
    df_train_alunos[col] = pd.to_numeric(df_train_alunos[col], errors='coerce')
    if col not in ['lat', 'lon']:
        df_test_alunos[col] = pd.to_numeric(df_test_alunos[col], errors='coerce')

In [32]:
df_train_alunos[df_train_alunos.duplicated()]

,ponto_id,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,lat,lon,indoor


In [33]:
df_test_alunos[df_test_alunos.duplicated()]

,ponto_id,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3


In [34]:
df_train_alunos.isnull().sum()

ponto_id    0
rssi_1_1    0
rssi_1_2    0
rssi_1_3    0
rssi_2_1    0
rssi_2_2    0
rssi_2_3    0
rssi_3_1    0
rssi_3_2    0
rssi_3_3    0
delay_1     0
delay_2     0
delay_3     0
lat         0
lon         0
indoor      0
dtype: int64

In [35]:
df_test_alunos.isnull().sum()

ponto_id    0
rssi_1_1    0
rssi_1_2    0
rssi_1_3    0
rssi_2_1    0
rssi_2_2    0
rssi_2_3    0
rssi_3_1    0
rssi_3_2    0
rssi_3_3    0
delay_1     0
delay_2     0
delay_3     0
dtype: int64

In [36]:
df_train_alunos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6775 entries, 0 to 6774
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ponto_id  6775 non-null   int64  
 1   rssi_1_1  6775 non-null   float64
 2   rssi_1_2  6775 non-null   float64
 3   rssi_1_3  6775 non-null   float64
 4   rssi_2_1  6775 non-null   float64
 5   rssi_2_2  6775 non-null   float64
 6   rssi_2_3  6775 non-null   float64
 7   rssi_3_1  6775 non-null   float64
 8   rssi_3_2  6775 non-null   float64
 9   rssi_3_3  6775 non-null   float64
 10  delay_1   6775 non-null   float64
 11  delay_2   6775 non-null   float64
 12  delay_3   6775 non-null   float64
 13  lat       6775 non-null   float64
 14  lon       6775 non-null   float64
 15  indoor    6775 non-null   bool   
dtypes: bool(1), float64(14), int64(1)
memory usage: 800.7 KB


In [37]:
df_test_alunos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2904 entries, 0 to 2903
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ponto_id  2904 non-null   int64  
 1   rssi_1_1  2904 non-null   float64
 2   rssi_1_2  2904 non-null   float64
 3   rssi_1_3  2904 non-null   float64
 4   rssi_2_1  2904 non-null   float64
 5   rssi_2_2  2904 non-null   float64
 6   rssi_2_3  2904 non-null   float64
 7   rssi_3_1  2904 non-null   float64
 8   rssi_3_2  2904 non-null   float64
 9   rssi_3_3  2904 non-null   float64
 10  delay_1   2904 non-null   float64
 11  delay_2   2904 non-null   float64
 12  delay_3   2904 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 295.1 KB


In [38]:
group_mapping = {
    'Grupo 1': ['rssi_1_1', 'rssi_1_2', 'rssi_1_3'],
    'Grupo 2': ['rssi_2_1', 'rssi_2_2', 'rssi_2_3'],
    'Grupo 3': ['rssi_3_1', 'rssi_3_2', 'rssi_3_3']
}

limit = 40

to_be_removed = list()

for group_name, group_columns in group_mapping.items():
    df_group = df_train_alunos[group_columns].copy()

    df_group['max_diff'] = df_group.max(axis=1) - df_group.min(axis=1)

    indexes_above_limit = df_group[df_group['max_diff'] > limit].index
    to_be_removed.extend(indexes_above_limit)

    print(f'Para o {group_name}')
    print(f'{len(indexes_above_limit)}/{len(df_group)} linhas estão com valores fora do limite (diferença máxima tolerada = {limit}).\n')

df_train_alunos = df_train_alunos.drop(to_be_removed)

Para o Grupo 1
0/6775 linhas estão com valores fora do limite (diferença máxima tolerada = 40).

Para o Grupo 2
9/6775 linhas estão com valores fora do limite (diferença máxima tolerada = 40).

Para o Grupo 3
3/6775 linhas estão com valores fora do limite (diferença máxima tolerada = 40).



In [39]:
df_bts.to_csv('input_data/02_pre_processed_files/bts.csv')
df_train_alunos.to_csv('input_data/02_pre_processed_files/meds_train_alunos.csv')
df_test_alunos.to_csv('input_data/02_pre_processed_files/meds_test_alunos.csv')

: 